In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import RandomForestRegressor

In [2]:
folder = "D:/Documents/datasets/CPP utec/"
df1_file = folder + "entrenamiento.csv"
df2_file = folder + "evaluacion.csv"

In [3]:
df1 = pd.read_csv(df1_file, na_values="?")
df2 = pd.read_csv(df2_file, na_values="?")

In [4]:
df1.drop('id', axis=1, inplace=True)
df2.drop('id', axis=1, inplace=True)

In [5]:
df1.shape, df2.shape

((144, 25), (61, 24))

In [6]:
frames = [df1, df2]

In [7]:
df = pd.concat(frames)

df

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,0,volkswagen rabbit,gas,std,four,sedan,fwd,front,108.0,184.5,...,140.0,mpfi,3.25,3.44,8.9,115.0,5666.0,19.0,25.0,13295.0
1,2,audi fox,gas,std,two,sedan,fwd,front,107.4,181.5,...,140.0,mpfi,3.25,3.44,8.9,115.0,5666.0,19.0,26.0,15250.0
2,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,95.3,172.8,...,134.0,mpfi,3.53,2.71,9.4,116.0,5151.0,21.0,28.0,13495.0
3,-1,buick opel isuzu deluxe,gas,std,four,sedan,rwd,front,124.4,207.4,...,241.0,mpfi,3.52,3.13,8.7,162.0,4893.0,16.0,19.0,34184.0
4,0,toyota corolla,diesel,std,four,hatchback,fwd,front,103.0,170.2,...,113.0,idi,3.33,3.39,23.5,59.0,4636.0,38.0,49.0,7788.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0,volkswagen dasher,gas,std,four,wagon,fwd,front,108.0,187.4,...,112.0,mpfi,3.25,3.44,9.4,92.0,5666.0,25.0,32.0,NaN
57,-2,volvo 145e (sw),gas,std,four,sedan,rwd,front,112.2,193.3,...,145.0,mpfi,3.85,3.19,9.9,119.0,5563.0,23.0,29.0,NaN
58,-2,volvo 244dl,gas,std,four,sedan,rwd,front,112.2,193.3,...,145.0,mpfi,3.85,3.19,9.9,119.0,5563.0,24.0,29.0,NaN
59,-1,volvo diesel,gas,turbo,four,wagon,rwd,front,112.2,193.3,...,134.0,mpfi,3.69,3.19,7.8,170.0,5254.0,17.0,23.0,NaN


In [8]:
# Get dummies
df = pd.get_dummies(df, columns=["CarName", "fueltype", "aspiration", "doornumber", "carbody", "drivewheel", "enginelocation", "enginetype", "cylindernumber", "fuelsystem"], prefix={"CarName": "carname", "fueltype": "fueltype", "aspiration": "aspiration", "doornumber": "doornumber", "carbody": "carbody", "drivewheel": "drivewheel", "enginelocation": "enginelocation", "enginetype": "enginetype", "cylindernumber": "cylindernumber", "fuelsystem": "fuelsystem"})

In [9]:
df_train = df.iloc[:144,:]
_eval = df.iloc[144:,:]

_eval = _eval.drop('price', axis=1)

In [10]:
y = df_train['price']
X = df_train.drop('price', axis=1)

#X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.25,stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.28)

In [11]:
pd.set_option('display.max_columns', None)

In [12]:
df_train.head(500)

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,carname_Nissan versa,carname_alfa-romero Quadrifoglio,carname_alfa-romero giulia,carname_alfa-romero stelvio,carname_audi 100 ls,carname_audi 100ls,carname_audi 4000,carname_audi 5000,carname_audi 5000s (diesel),carname_audi fox,carname_bmw 320i,carname_bmw x1,carname_bmw x3,carname_bmw x4,carname_bmw x5,carname_bmw z4,carname_buick century,carname_buick century luxus (sw),carname_buick century special,carname_buick electra 225 custom,carname_buick opel isuzu deluxe,carname_buick regal sport coupe (turbo),carname_buick skyhawk,carname_buick skylark,carname_chevrolet impala,carname_chevrolet monte carlo,carname_chevrolet vega 2300,carname_dodge challenger se,carname_dodge colt (sw),carname_dodge colt hardtop,carname_dodge coronet custom,carname_dodge coronet custom (sw),carname_dodge d200,carname_dodge dart custom,carname_dodge monaco (sw),carname_dodge rampage,carname_honda accord,carname_honda accord cvcc,carname_honda accord lx,carname_honda civic,carname_honda civic (auto),carname_honda civic 1300,carname_honda civic 1500 gl,carname_honda civic cvcc,carname_honda prelude,carname_isuzu D-Max,carname_isuzu D-Max V-Cross,carname_isuzu MU-X,carname_jaguar xf,carname_jaguar xj,carname_jaguar xk,carname_maxda glc deluxe,carname_maxda rx3,carname_mazda 626,carname_mazda glc,carname_mazda glc 4,carname_mazda glc custom,carname_mazda glc custom l,carname_mazda glc deluxe,carname_mazda rx-4,carname_mazda rx-7 gs,carname_mazda rx2 coupe,carname_mercury cougar,carname_mitsubishi g4,carname_mitsubishi lancer,carname_mitsubishi mirage,carname_mitsubishi mirage g4,carname_mitsubishi montero,carname_mitsubishi outlander,carname_mitsubishi pajero,carname_nissan clipper,carname_nissan dayz,carname_nissan fuga,carname_nissan gt-r,carname_nissan juke,carname_nissan kicks,carname_nissan latio,carname_nissan leaf,carname_nissan note,carname_nissan nv200,carname_nissan otti,carname_nissan rogue,carname_nissan teana,carname_nissan titan,carname_peugeot 304,carname_peugeot 504,carname_peugeot 504 (sw),carname_peugeot 505s turbo diesel,carname_peugeot 604sl,carname_plymouth cricket,carname_plymouth duster,carname_plymouth fury gran sedan,carname_plymouth fury iii,carname_plymouth satellite custom (sw),carname_plymouth valiant,carname_porcshce panamera,carname_porsche boxter,carname_porsche cayenne,carname_porsche macan,carname_renault 12tl,carname_renault 5 gtl,carname_saab 99e,carname_saab 99gle,carname_saab 99le,carname_subaru,carname_subaru baja,carname_subaru brz,carname_subaru dl,carname_subaru r1,carname_subaru r2,carname_subaru trezia,carname_subaru tribeca,carname_toyota carina,carname_toyota celica gt,carname_toyota celica gt liftback,carname_toyota corolla,carname_toyota corolla 1200,carname_toyota corolla 1600 (sw),carname_toyota corolla liftback,carname_toyota corolla tercel,carname_toyota corona,carname_toyota corona hardtop,carname_toyota corona liftback,carname_toyota corona mark ii,carname_toyota cressida,carname_toyota mark ii,carname_toyota starlet,carname_toyota tercel,carname_toyouta tercel,carname_vokswagen rabbit,carname_volkswagen 1131 deluxe sedan,carname_volkswagen 411 (sw),carname_volkswagen dasher,carname_volkswagen model 111,carname_volkswagen rabbit,carname_volkswagen rabbit custom,carname_volkswagen super beetle,carname_volkswagen type 3,carname_volvo 144ea,carname_volvo 145e (sw),carname_volvo 244dl,carname_volvo 245,carname_volvo 246,carname_volvo 264gl,carname_volvo diesel,carname_vw dasher,carname_vw rabbit,fueltype_diesel,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon,drivewheel_4wd,drivewheel_fwd,drivewheel_rwd,enginelocation_front,enginelocation_rear,enginetype_dohc,enginetype_dohcv,enginetype_l,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_roto

In [13]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 0 to 60
Data columns (total 200 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   symboling                                int64  
 1   wheelbase                                float64
 2   carlength                                float64
 3   carwidth                                 float64
 4   carheight                                float64
 5   curbweight                               float64
 6   enginesize                               float64
 7   boreratio                                float64
 8   stroke                                   float64
 9   compressionratio                         float64
 10  horsepower                               float64
 11  peakrpm                                  float64
 12  citympg                                  float64
 13  highwaympg                               float64
 14  price                     

In [14]:
rfr_param_grid = {
    'bootstrap': [True, False], 
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None], 
    'max_features': ['auto', 'log2', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 20, 40]
}

In [75]:
rfr = RandomForestRegressor(bootstrap=False,
                             max_depth=15,
                             max_features='sqrt',
                             min_samples_leaf=1,
                             min_samples_split=2,
                             n_estimators=10)

rfr.fit(X_train, y_train)

print ("test score", rfr.score(X_test, y_test))
print("model predict", rfr.predict(_eval))

predict = rfr.predict(_eval)
for ind, p in enumerate(predict):
    print(f'{ind},{round(p, 2)}')

AttributeError: 'RandomForestRegressor' object has no attribute 'roc'

In [16]:
rfr = RandomForestRegressor(random_state = 1)

g_search = GridSearchCV(
    estimator = rfr,
    param_grid = rfr_param_grid,
    cv = 3,
    n_jobs = 1,
    verbose = 1,
    return_train_score=True)

g_search.fit(X_train, y_train)

Fitting 3 folds for each of 9504 candidates, totalling 28512 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 28512 out of 28512 | elapsed:  8.6min finished


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=1), n_jobs=1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100,
                                       None],
                         'max_features': ['auto', 'log2', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 15, 20, 40]},
             return_train_score=True, verbose=1)

In [17]:
print(g_search.best_params_)
best_grid = g_search.best_estimator_
print(best_grid.score(X_test, y_test))
print(best_grid.predict(_eval))

{'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 15}
0.9221596297109274
[13496.26666667 11109.4        18280.2        17837.13333333
 37618.8         5685.66666667  5924.13333333  6549.
  6685.93333333  6776.13333333  7239.4         7996.8
  8490.46666667 10964.93333333 41725.86666667 39453.13333333
  6080.93333333  7333.66666667  8688.53333333  8917.4
 16583.66666667 10888.2        26923.62226667 30800.63333333
 24708.5         8357.8         9189.         14421.
  9418.73333333 10338.73333333 10603.13333333  7642.23333333
  6103.86666667  6462.66666667  6879.          7174.93333333
  6668.13333333 17077.33333333 16830.26666667 16830.26666667
 16830.26666667 15335.66666667 19109.4778      6474.33333333
 16281.61113333 14179.86666667  7573.06666667  8460.53333333
 10843.8        10462.66666667  7302.          7145.43333333
  7920.46666667 10905.66666667 13200.06666667  8899.2
 11386.06666667 15668.53333333 1649

In [18]:
rfr_param_grid2 = {}
rfr1 = RandomForestRegressor(random_state=1, n_jobs=1, bootstrap=False, max_depth=20, max_features='sqrt', min_samples_leaf=1,min_samples_split=2, n_estimators=1)

rfr1.fit(X_train, y_train)

print ("test score", rfr1.score(X_test, y_test))
print("model predict", rfr1.predict(_eval))

test score 0.8607091688786034
model predict [13495.  11259.  23875.  23875.  41315.   6189.   6189.   6692.   6692.
  5348.   7957.   8845.   6918.  11259.  41315.  41315.   6338.   6488.
  8495.   8495.  19045.  16900.  31400.5 17199.  19045.   7957.   8845.
 12629.   8845.  10198.  10198.   5348.   5499.   5499.   6849.   7349.
  5499.  16500.  21485.  21485.  21485.  13860.  21485.   6692.  12629.
 15510.   7957.  11694.  11694.   9995.   8358.   8916.5  6938.   8449.
 11549.   9995.   8495.  13415.  16515.  19045.  13415. ]


In [19]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
model.fit(X_train, y_train)


print ("test score", model.score(X_test, y_test))
print("model predict", model.predict(_eval))

test score 0.892368389631994
model predict [13495.     9295.    19045.    17859.167 31400.5    5572.     6189.
  6189.     6692.     6575.     6849.     6938.     6938.    11694.
 31400.5   34184.     6189.     6189.     8495.     8845.    13495.
 10698.    31600.    34028.    17199.     8845.    11595.    14869.
 10295.    10295.     9988.     6575.     6849.     7349.     7299.
  7349.     7349.    17199.    19045.    22470.    22470.    16900.
 22470.     6189.    16500.    15510.     6785.     6785.     8845.
 10795.     8358.     6575.     8058.    11199.    11248.     7775.
  9295.    20970.    20970.    16515.    20970.   ]


In [20]:
dtm = DecisionTreeRegressor(max_depth=4,
                           min_samples_split=5,
                           max_leaf_nodes=7)

param_grid = {"criterion": ["mse", "mae"],
              "min_samples_split": [2, 20, 40],
              "max_depth": [2, 6, 8],
              "min_samples_leaf": [10, 20, 60],
              "max_leaf_nodes": [5, 20, 40],
              }

grid_cv_dtm = GridSearchCV(dtm, param_grid, cv=5)

grid_cv_dtm.fit(X_train,y_train)

print("R-Squared::{}".format(grid_cv_dtm.best_score_))
print("Best Hyperparameters::\n{}".format(grid_cv_dtm.best_params_))

R-Squared::0.6160727452704788
Best Hyperparameters::
{'criterion': 'mae', 'max_depth': 2, 'max_leaf_nodes': 5, 'min_samples_leaf': 10, 'min_samples_split': 2}
